#### 계좌새창, 보증인새창으로 신용회복 업무대장 data시트 작성

In [2]:
import pandas as pd
import os 
from os.path import join
import pickle
import re


def 파일명(폴더:str, 키워드:str, 부정키워드="") :
    file_list = os.listdir(폴더)
    if 부정키워드 == "" : 
        fn = [file for file in file_list if re.search(키워드, file)]
    else : 
        fn = [file for file in file_list if (re.search(키워드, file)!=None) & (re.search(부정키워드,file)==None)]
    if len(fn) == 1 :
        return join(폴더,fn[0])
    else :
        print("조건을 만족하는 파일이 둘 이상입니다.")

In [7]:
##############################
company = "대성"
base_date = "240430"
##############################
wd = join(r"D:\3.자산\전산 dataset",company, base_date,"pkl")
fn_account = 파일명(wd, "계좌조회새창")
fn_grt = 파일명(wd, "보증인새창", "새보증인상태")

account = pd.read_pickle(join(wd, fn_account))
grt = pd.read_pickle(join(wd, fn_grt))

In [8]:
# 솔림 - 대성 - 솔림 중 새로 채무자키 부여한 거 제거
if company == "솔림" : 
    삭제할채무자키 = ["20434444","20434445","20428465","20413796","20552818","20495098","20495130"]

    account = account.query('채무자키 not in(@삭제할채무자키)')
    grt = grt.query('채무자키 not in(@삭제할채무자키)')
else : pass

In [9]:
# data 작성
data_acc = account.loc[:,["채무자키", "채무자주민번호", "계좌번호", "채무자명", "계좌키"]].rename(columns={"채무자주민번호":"주민번호", "채무자명":"이름"})
data_acc["보증인키"] = ""
data_acc["채무구분"] = "차주"

data_grt = grt.loc[:,["채무자키","주민번호인","계좌번호", "보증인성명인", "계좌키", "보증인키"]].rename(columns={"주민번호인":"주민번호","보증인성명인":"이름"})
data_grt["채무구분"] = "보증인"

data = pd.concat([data_acc,data_grt], axis=0, ignore_index=True)

# 주민번호 없는 거 index로 대체---, * 있는 거는 모두 index로 대체할까하다 중복값 없길래 말았음
data.loc[data[data.주민번호 == ""].index, "이름"] = str(data[data.주민번호 == ""].index)

# 주민번호별 고유채무자키수
고유채무자키수 = data.groupby('주민번호')['채무자키'].nunique()
# 주민번호별 채무자키의 고유한 개수를 "고유채무자키수" 열에 매핑합니다.
data['고유채무자키수'] = data['주민번호'].map(고유채무자키수)

# 채무자키별 고유계좌수
고유계좌수 = data.groupby('채무자키')['계좌키'].nunique()
data["채무자키별고유계좌수"] = data['채무자키'].map(고유계좌수)

# 풀, 신탁, 수취계좌 (안씀)
data[["풀", "신탁", "수취계좌"]] = ""

# 주민+계좌번호별 고유 계좌수
data["주민+계좌"] = data["주민번호"] + data["계좌번호"]
주민계좌별고유계좌수 = data.groupby('주민+계좌')['계좌키'].nunique()
data["주민+계좌별 고유 계좌키수"] = data["주민+계좌"].map(주민계좌별고유계좌수)

# 원채권사, 매각사구분 - 계좌키 중복이어도 문제 없는 거 확인함
data = data.merge(account.loc[:,["계좌키", "원채권사", "매각사구분"]], on="계좌키", how="left")

# 주민번호 총 계좌수
주민번호고유계좌수 = data.groupby('주민번호')['계좌키'].nunique()
data["주민번호총계좌수"] = data["주민번호"].map(주민번호고유계좌수)

In [10]:
data.to_excel(join(r"C:\Users\DATA\Desktop", company+"data.xlsx"), index=False)